In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# First try at Movie Recommendation System.

My first try at mmakking a movie recommendation sysytem.

With the rise in Data collection the use of Recommendation System has become very frequent.

Recommendation Systems are a type of **information filtering** system as they **improve** the **quality of search**.

Ratings are used to predict the preferences of  a viewer.

Amazon uses such sysytems to recommend products to its customers.

And the USP of both Netflix and Spotify is their recommendation engines.

# TYPES OF RECOMENDER SYSTEMS:
* Collaborative Recommender system
* Content-based recommender system
* Demographic based recommender system
* Utility based recommender system
* Knowledge based recommender system 
* Hybrid recommender system

# Loading Data

In [ ]:
data1=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
data2=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

Lets see whats in data1

In [ ]:
data1.head()

ANd in data2

In [ ]:
data2.head()

Merging the data sets.

In [ ]:
data1.columns=['id','title','cast','crew']
data3=data2.merge(data1,on='id')

In [ ]:
data3.head()

# Lets try Content Based Filtering

It’s mainly classified as an outgrowth and continuation of information filtering research. In this system, the objects are mainly defined by their associated features. A content-based recommender learns a profile of the new user’s interests based on the features present, in objects the user has rated. It’s basically a keyword specific recommender system here keywords are used to describe the items. Thus, in a content-based recommender system the algorithms used are such that it recommends users similar items that the user has liked in the past or is examining currently.


The way we are going to do this is by using metadata like the Actors,Director,Genre,Plot Keywords

In [ ]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    data3[feature] = data3[feature].apply(literal_eval)

In [ ]:
#geting the name of the director
def director(n):
    for i in n:
        if i['job']=="Director":
            return i['name']
    return np.nan
    

In [ ]:
#create a function to return only 3 elements
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
#defining new director,cast,crew,keyword features
data3['director']=data3['crew'].apply(director)
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    data3[feature] = data3[feature].apply(get_list)

In [ ]:
#checking the new features
data3[['title_x','director','cast','crew','keywords','genres']].head()

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    data3[feature] = data3[feature].apply(clean_data)

We are now in a position to create our "metadata soup", which is a string that contains all the metadata that we want to feed to our vectorizer (namely actors, director and keywords).

In [ ]:
def meta_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
data3['soup'] = data3.apply(meta_soup, axis=1)

We use the **CountVectorizer()** instead of **TF-IDF**. This is because we do not want to down-weight the presence of an actor/director if he or she has acted or directed in relatively more movies. It doesn't make much intuitive sense

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data3['soup'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
data3 = data3.reset_index()
indices = pd.Series(data3.index, index=data3['title_y'])

Now we create the recommendation function

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data3['title_x'].iloc[movie_indices]

In [ ]:
get_recommendations("Spectre")

In [ ]:
get_recommendations("Apocalypse Now")

In [ ]:
get_recommendations("Before Sunrise")

# Let's try Collaborative Filtering

The content based engine suffers from some severe limitations. It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.

**Content-based** approach requires a good amount of information of items’ own features, rather than using users’ interactions and feedbacks. For example, it can be movie attributes such as genre, year, director, actor etc., or textual content of articles that can extracted by applying Natural Language Processing. **Collaborative Filtering**, on the other hand, doesn’t need anything else except users’ historical preference on a set of items. Because it’s based on historical data, the core assumption here is that the users who have agreed in the past tend to also agree in the future. In terms of user preference, it usually expressed by two categories. **Explicit Rating**, is a rate given by a user to an item on a sliding scale, like 5 stars for Titanic. This is the most direct feedback from users to show how much they like an item. Implicit Rating, suggests users preference indirectly, such as page views, clicks, purchase records, whether or not listen to a music track, and so on. In this article, I will take a close look at collaborative filtering that is a traditional and powerful tool for recommender systems.

**User Based Collaborative Filtering**
We have an n × m matrix of ratings, with user uᵢ, i = 1, ...n and item pⱼ, j=1, …m. Now we want to predict the rating rᵢⱼ if target user i did not watch/rate an item j. The process is to calculate the similarities between target user i and all other users, select the top X similar users, and take the weighted average of ratings from these X users with similarities as weights.
![](https://miro.medium.com/max/700/1*mM089Lta5X6zkUkULcO9aA.png)

While different people may have different baselines when giving ratings, some people tend to give high scores generally, some are pretty strict even though they are satisfied with items. To avoid this bias, we can subtract each user’s average rating of all items when computing weighted average, and add it back for target user, shown as below.
![](https://miro.medium.com/max/700/1*gLbwJts3g_v2TbPRhFoNfA.png)

**Item-based Collaborative Filtering**

In Item-based CF, we say two items are similar when they received similar ratings from a same user. Then, we will make prediction for a target user on an item by calculating weighted average of ratings on most X similar items from this user. One key advantage of Item-based CF is the stability which is that the ratings on a given item will not change significantly overtime, unlike the tastes of human beings.
                        ![](https://miro.medium.com/max/700/1*dPzd5-dScFplypBGeSwgUw.png)

**Single Value Decomposition**

One way to handle the scalability and sparsity issue created by CF is to leverage a latent factor model to capture the similarity between users and items. Essentially, we want to turn the recommendation problem into an optimization problem. We can view it as how good we are in predicting the rating for items given a user. One common metric is Root Mean Square Error (RMSE). The lower the RMSE, the better the performance.

Now talking about latent factor you might be wondering what is it ?It is a broad idea which describes a property or concept that a user or an item have. For instance, for music, latent factor can refer to the genre that the music belongs to. SVD decreases the dimension of the utility matrix by extracting its latent factors. Essentially, we map each user and each item into a latent space with dimension r. Therefore, it helps us better understand the relationship between users and items as they become directly comparable. The below figure illustrates this idea.
![](https://kevinkolcheck.com/wp-content/uploads/2017/12/latent-factors.png)

To implement **Single Value Decomposition** were going to use the **surprise** library.


In [ ]:
from surprise import Dataset,SVD,Reader
from surprise.model_selection import cross_validate
reader = Reader()
ratings = pd.read_csv('../input/the-movies-dataset/ratings_small.csv')
ratings.head()

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd=SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Lets see what user no.10 has rated wach movie

In [ ]:
ratings[ratings['userId'] == 10]

In [ ]:
svd.predict(10, 2995, 3)

For movie with ID 2995, we get an estimated prediction of 2.467. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.